In [12]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
import time

In [21]:
X, y = make_regression(n_features=2, random_state=0, shuffle=False, n_samples=100)
model = RandomForestRegressor(max_depth=2, n_estimators=10, random_state=0)
model.fit(X, y)
#for i in range(0,5):
print(model.predict([[0, 0]]))
    #time.sleep(5)

[38.23613716]


In [23]:
from sklearn import tree
import pickle

In [24]:
X = [[0, 0, 2], [1, 1, 3], [3, 1, 3]]
y = [0, 1, 2]

In [25]:
model = tree.DecisionTreeClassifier()

In [26]:
model.fit(X, y)

DecisionTreeClassifier()

In [27]:
with open("dt.pkl", "wb") as file:
    pickle.dump(model, file)

In [30]:
print(model.predict([[1, 1, 6]]))

[1]


In [31]:
params = model.__dict__

In [32]:
with open("./dt_params.pkl", "wb") as param_file:
        pickle.dump(params, param_file)

In [36]:
import json
import pickle
from sklearn import tree
import numpy as np
from flask import Flask, jsonify, request

In [37]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

app = Flask(__name__)

In [39]:
@app.route("/predict-with-params", methods=['POST'])
def predict_loading_params():
    with open("dt.pkl", "rb") as file:
        model: tree.DecisionTreeClassifier = pickle.load(file)
        # Loading params from param store
        param_file = open("dt_params.pkl", "rb")
        params = pickle.load(param_file)
        print(params)
        model.__dict__ = params
        X = json.loads(request.data)
        print(X)
        response = model.predict(X)
        return json.dumps(response, cls=NumpyEncoder)

@app.route("/predict-with-full-model", methods=['POST'])
def predict_with_full_model():
    with open("dt.pkl", "rb") as file:
        model: tree.DecisionTreeClassifier = pickle.load(file)
        X = json.load(request.data)
        print(X)
        response = model.predict(X)
        return json.dumps(response, cls=NumpyEncoder)
app.run()

AssertionError: View function mapping is overwriting an existing endpoint function: predict_loading_params